# Project 4

### Goals/notes
* Focus on Uganda, families, 2011

In [2]:
!pip install CFEDemands --upgrade
!pip install ConsumerDemands
!pip install oauth2client
!pip install eep153_tools
!pip install gnupg
#!pip install dvc

  Using cached CFEDemands-0.4.1-py2.py3-none-any.whl (39 kB)
  Using cached ConsumerDemands-0.3.dev0-py2.py3-none-any.whl (12 kB)
  Using cached oauth2client-4.1.3-py2.py3-none-any.whl (98 kB)
  Using cached eep153_tools-0.11-py2.py3-none-any.whl (4.4 kB)
Processing ./.cache/pip/wheels/20/7e/30/7d702acd6a1e89911301cd9dbf9cb9870ca80c0e64bc2cde23/gnupg-2.3.1-py3-none-any.whl


In [3]:
from eep153_tools.sheets import read_sheets
import numpy as np
import pandas as pd

### Data

In [4]:
InputFiles = {'Expenditures':('1mZcHvSD55wGCicdqxbEnioE3AOXM_pVb6zVwHhtBzSk','Expenditures'),
              'Consumption':('1RT5AfbZbWx-CmNaOHLp7aD9qEjgjiBv8lZmPexuKEWg','Consumption'),
              'HH Characteristics':('1mZcHvSD55wGCicdqxbEnioE3AOXM_pVb6zVwHhtBzSk','HH Characteristics'),
              'FCT': ('1DjpLsyhUbQ-giGcTS_GMrDuW_wG3VDrYH1p-FanKxxs','Codes')}

#### Function to read in sheets:

In [5]:
def get_clean_sheet(key,sheet=None):
    dfs = read_sheets(key)
    if sheet is not None:
        df = dfs[sheet]
    else:
        df = dfs.values()[0] 
    df.columns = [c.strip() for c in df.columns.tolist()]
    df = df.loc[:,~df.columns.duplicated(keep='first')]   
    df = df.drop([col for col in df.columns if col.startswith('Unnamed')], axis=1)
    df = df.loc[~df.index.duplicated(), :]
    return df

#### Expenditure Table

In [36]:
x = get_clean_sheet(InputFiles['Expenditures'][0],
                    sheet=InputFiles['Expenditures'][1])

if 'm' not in x.columns:
    x['m'] = 1

x = x.set_index(['j','t','m'])
x.columns.name = 'i'

x = x.apply(lambda x: pd.to_numeric(x,errors='coerce'))
x = x.replace(0,np.nan)

# We only want data from 2011, as that is our focus year:
x_2011 = x[x.index.isin([2011], level=1)]

# Log of expenditure table (x); 
# log table (y) will be used for later estimation:
y_2011 = np.log(x_2011)

Key available for students@eep153.iam.gserviceaccount.com.


#### Household Characteristics Table

In [39]:
z = get_clean_sheet(InputFiles['HH Characteristics'][0],
                    sheet=InputFiles['HH Characteristics'][1])

if 'm' not in z.columns:
    z['m'] = 1

z = z.set_index(['j','t','m'])
z.columns.name = 'k'

z = z.apply(lambda x: pd.to_numeric(x,errors='coerce'))

z_2011 = z[z.index.isin([2011], level=1)]

Key available for students@eep153.iam.gserviceaccount.com.


#### Consumption Quantities Table

In [8]:
q = get_clean_sheet(InputFiles['Consumption'][0],
                    sheet=InputFiles['Consumption'][1])

if 'm' not in q.columns:
    q['m'] = 1

q = q.set_index(['j','t','m','u'])
q.columns.name = 'i'

q = q.apply(lambda x: pd.to_numeric(x,errors='coerce'))
q = q.replace(0,np.nan)

Key available for students@eep153.iam.gserviceaccount.com.


#### Food Conversion Table

In [9]:
fct = get_clean_sheet(InputFiles['FCT'][0],sheet=InputFiles['FCT'][1])

fct.index.name = 'i'
foods = fct['Food description']
fct = fct.apply(lambda x: pd.to_numeric(x,errors='coerce'))
fct['Food description'] = foods
fct = fct.set_index('Food description')

Key available for students@eep153.iam.gserviceaccount.com.


## Population

* Compare households **in the same income bracket** with **different numbers of children** to understand how demand of nutrients changes per child added to a household
    - We don’t have income levels, we only have household expenditures
    - Does this cause a problem, like if a family of 6 in one income bracket has a similar household expenditure to a family of 6 in a higher income bracket???
    - Be really picky about households, make sure they’re actually very similar
* We might want to take **a couple households with the same number of children each and find an average** (like an average for a household with 2 children, an average for a household with 5 children) and compare averages


Ideas:
* Definitely create filters
    - Children TOTAL filter
    - Adult TOTAL filter
    - Doesn't matter what the breakdown is, as long as children=children and adult=adult for every household in that particular dataset
    - We can call any combo of age breakdowns with this system
* Look up income data in a separate place? Will it even be household specific tho, to the same household id's that we have here?

In [41]:
# Make two columns for filtering; 
# A total count of children and a total count of adults

z_count = z_2011
z_count["Children"], z_count["Adults"] = [(z_2011["M 0-3"] + z_2011["F 0-3"] + 
                                           z_2011["M 4-8"] + z_2011["F 4-8"] +
                                           z_2011["M 9-13"] + z_2011["F 9-13"] +
                                           z_2011["M 14-18"] + z_2011["F 14-18"]),
                                          (z_2011["M 19-30"] + z_2011["F 19-30"] + 
                                           z_2011["M 31-50"] + z_2011["F 31-50"] +
                                           z_2011["M 51+"] + z_2011["F 51+"])]

/tmp/ipykernel_64/1615739697.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  z_count["Children"], z_count["Adults"] = [(z_2011["M 0-3"] + z_2011["F 0-3"] +


In [63]:
# Do we only want households that have children?

filter_kid = z_count["Children"]!=0

In [64]:
# Here are households that have 1 or more children:
z_kids = z_count[filter_kid]

In [65]:
# Looking at some household breakdowns to decide which adult/child ratios might be good to investigate:

df_new = z_kids[['Children', 'Adults']]
df_new.head(60)

,,k,Children,Adults
j,t,m,,
1013000201,2011,1,5,2
1013000202,2011,1,2,1
1013000204,2011,1,3,2
1013000212,2011,1,3,3
1021000102,2011,1,2,4
1021000111,2011,1,4,5
1021000113,2011,1,5,4
1021000201,2011,1,6,2
1021000203,2011,1,5,3


Now what dataframes should we create to focus on? We can do any combination of adults and children in a house;
* Focus on houses with just 2 adults, as a control measure (and because it's overwhelmingly common)
    - But that could be a bit biased; there are some households with many adults, perhaps multi-family households. These could be more likely to be low-income (living together to support each other), and we are just omitting them from the get-go. Something to note !
* Look at the difference from 2 kids (1 dataset) to 3 (2 datasets), 3 kids to 4 (3 datasets)... perhaps 4 to 5 (4 datasets) as well. Yeah 4 dataframes is manageable
* Do analysis regardless of income level? Because we don't know it? Get an average 

In [85]:
# 1 kid filter:
filter1 = z_kids["Children"]==1

# 2 kids filter:
filter2 = z_kids["Children"]==2

# 3 kids filter:
filter3 = z_kids["Children"]==3

# 4 kids filter:
filter4 = z_kids["Children"]==4

# 5 kids filter:
filter5 = z_kids["Children"]==5



# Only 2 adults filter:
filter_adult = z_kids["Adults"]==2

In [86]:
# Use filters in different combinations to get 4 different dataframes:

one_kid = z_kids[filter1 & filter_adult]

two_kid = z_kids[filter2 & filter_adult]

three_kid = z_kids[filter3 & filter_adult]

four_kid = z_kids[filter4 & filter_adult]

five_kid = z_kids[filter5 & filter_adult]

### 1 child, 2 adult households:

In [87]:
print('Number of 1 child households: ', len(one_kid))

Number of 1 child households:  146


### 2 child, 2 adult households:

In [88]:
print('Number of 2 child households: ', len(two_kid))

Number of 2 child households:  192


### 3 child, 2 adult households:

In [89]:
print('Number of 3 child households: ', len(three_kid))

Number of 3 child households:  227


### 4 child, 2 adult households:

In [90]:
print('Number of 4 child households: ', len(four_kid))

Number of 4 child households:  218


### 5 child, 2 adult households:

In [91]:
print('Number of 3 child households: ', len(five_kid))

Number of 3 child households:  185


In [ ]:
# Now income? or no? 

## Estimate Demand System

In [10]:
# Create reduced form
import cfe

result = cfe.Result(y=y,z=z)

Missing dependencies for OracleDemands.


In [11]:
# run this to be able to run the next few lines:
result.get_reduced_form()

### Delta Values

In [11]:
result.delta.to_dataframe().unstack('k')

delta                                \
k                                      M 0-3     M 4-8    M 9-13   M 14-18   
i                                                                            
beans dry                          -0.020105  0.091101  0.069991  0.131634   
beans fresh                         0.003502  0.138232  0.049423  0.043303   
beef                                0.012622  0.025357  0.076339  0.081080   
bread                              -0.010931 -0.010405  0.039982  0.015581   
cabbages                            0.014337 -0.007694  0.035684  0.098675   
cassava dry/flour                   0.240597  0.163090  0.237445  0.090692   
cassava fresh                      -0.003535  0.006866  0.158522  0.094510   
chicken                             0.005440 -0.043764  0.015580  0.035094   
coffee                              0.022706 -0.057133  0.028877  0.008619   
cooking oil                        -0.029436 -0.006941  0.021453  0.052854   
dodo                               -0.064294  0.010363  0.145474  0.102694   
dry/smoked fish                    -0.037726  0.043358  0.094554  0.125807   
eggs                               -0.071013  0.041225  0.006218  0.000555   
expenditure in restaurants on food -0.094559  0.037472  0.151278  0.025689   
fresh fish                          0.020568  0.080817  0.028284  0.073189   
fresh milk                         -0.001583 -0.002583 -0.014292 -0.023467   
goat meat                          -0.052198  0.101357  0.009729  0.055356   
ground nuts pounded                 0.033751  0.022292  0.009320  0.044031   
irish potatoes                      0.086792  0.069802  0.115015  0.128570   
maize cobs                         -0.064494 -0.044831  0.043807  0.182292   
maize flour                         0.009996  0.112082  0.142091  0.075552   
mangoes                            -0.012302  0.032207  0.053649 -0.052200   
margarine,butter                   -0.015707  0.022894 -0.081790 -0.020390   
matooke_1                          -0.017372  0.179616  0.051870  0.053682   
millet                              0.096938  0.067989  0.001464  0.142472   
onions                             -0.051036 -0.027855  0.030258  0.048141   
oranges                            -0.050278  0.097141  0.035387  0.028289   
other vegetables                    0.012330 -0.046587  0.078986  0.052800   
passion fruits                      0.007164 -0.067459  0.028914 -0.094513   
rice                                0.041959  0.089870  0.105172  0.099908   
salt                                0.072773  0.065634  0.082439  0.087910   
soda                               -0.011965 -0.063974  0.055411 -0.049988   
sugar                               0.012851  0.059009  0.045291  0.091241   
sweet bananas                       0.011865  0.005062  0.000327  0.025421   
sweet potatoes fresh                0.110313  0.127481  0.137242  0.100394   
tea                                -0.012445  0.016257  0.036033  0.041242   
tomatoes                           -0.040840 -0.022914  0.018166  0.050890   

                                                                            \
k                                    M 19-30   M 31-50     M 51+     F 0-3   
i                                                                            
beans dry                           0.092998  0.126748  0.192872  0.035235   
beans fresh                         0.140382  0.256491  0.353082  0.098333   
beef                                0.114490  0.197677  0.197012  0.004936   
bread                               0.133572  0.290906  0.142049  0.021215   
cabbages                            0.061728  0.060672  0.079951  0.038930   
cassava dry/flour                   0.106282  0.175328  0.190029  0.068200   
cassava fresh                       0.057267  0.043634  0.019687  0.003492   
chicken                             0.125162  0.132872  0.148593 -0.003726   
coffee                              0.098526  0.060754  0.42

### Alpha Values

In [12]:
result.a.to_dataframe().unstack('i')

a                                                              \
i      beans dry beans fresh      beef     bread  cabbages cassava dry/flour   
t    m                                                                         
2005 1  6.664045    6.166651  7.596403  6.531700  5.987894          7.000972   
2009 1  7.177776    6.660552  8.087632  7.200124  6.284722          7.403977   
2010 1  7.361983    6.874905  8.165703  7.073217  6.369316          7.427036   
2011 1  7.436129    7.184067  8.397829  7.388194  6.608653          7.770406   

                                                      ...                   \
i      cassava fresh   chicken    coffee cooking oil  ... other vegetables   
t    m                                                ...                    
2005 1      6.561716  8.220995  4.586281    6.181261  ...         5.622420   
2009 1      6.985613  8.733654  4.799347    6.605940  ...         6.134787   
2010 1      6.993906  8.798743  4.734974    6.763708  ...         6.100634   
2011 1      7.339227  9.207515  5.074747    6.940089  ...         6.475021   

                                                                             \
i      passion fruits      rice      salt      soda     sugar sweet bananas   
t    m                                                                        
2005 1       6.412390  6.868322  4.370730  7.055358  6.854877      5.973993   
2009 1       6.691317  7.375543  5.028736  7.325378  7.196572      6.407480   
2010 1       6.631896  7.440398  4.963925  7.503323  7.370396      6.422730   
2011 1       6.896860  7.743197  5.120522  7.765180  7.461896      6.816289   

                                                 
i      sweet potatoes fresh       tea  tomatoes  
t    m                                           
2005 1             6.627925  4.610117  5.958305  
2009 1             7.068341  4.678707  6.417172  
2010 1             7.447756  4.683057  6.427528  
2011 1             7.318681  4.936173  6.622662  

[4 rows x 37 columns]

### Beta Values

In [13]:
result.get_beta(as_df=True)

i
beans dry                             0.255622
beans fresh                           0.381523
beef                                  0.384351
bread                                 0.568417
cabbages                              0.357247
cassava dry/flour                     0.124855
cassava fresh                         0.313435
chicken                               0.322282
coffee                                0.632619
cooking oil                           0.483203
dodo                                  0.452957
dry/smoked fish                       0.351626
eggs                                  0.420556
expenditure in restaurants on food    0.460684
fresh fish                            0.412433
fresh milk                            0.567437
goat meat                             0.360544
ground nuts pounded                   0.367467
irish potatoes                        0.388275
maize cobs                            0.356432
maize flour                           0.285216
mangoes    

## Nutritional Adequacy Stuff

Note: All this code was written by someone now in another group: do we want to rewrite?? Not sure. Yes I might

*Deliverables:*
* **[A] Construct Nutrient System:** If you can describe quantities demanded as a function of prices, budgets, and household characteristics, and map quantities into nutrients, then you can also describe a system of nutrients as a function of the same variables

* **[A] Nutritional challenges & Policy Goal:** Compare predicted nutrition to recommended nutrition. Establish some criteria for deciding what the biggest nutritional challenges are for this population. For example, what share of households is Calorie deficient? Is protein deficient? Is Vitamin A deficient? Note that different criteria are possible, and there is no one “right” criterion. Characterize the nutritional challenges you observe in your data, and construct a policy goal to address some of these challenges (e.g., reduce the proportion of households that are protein deficient by half). 


In [13]:
foods = result.i.to_dataframe()
nutrients = fct[fct.index.isin(list(foods.index.values))]
q_items = q.loc[:,list(foods.index.values)]
q_items['Sum'] = q_items.sum(axis=1)
q_items = q_items[q_items['Sum']>0]
q_items = q_items.drop('Sum',axis=1)

In [14]:
diet_req = pd.read_csv("dietary_req.csv").drop(['Source'],axis=1)
dietreq = diet_req.set_index('Nutrition')
dietreq.iloc[len(dietreq)-1,:] = dietreq.iloc[len(dietreq)-1,:] * -1
diet_req = diet_req[diet_req['Nutrition'].isin(['Energy','Protein','Calcium, Ca','Iron, Fe', 'Magnesium, Mg','Vitamin A, RAE','Vitamin B-12','Vitamin B-6','Vitamin C, total ascorbic acid','Zinc, Zn','Vitamin E (alpha-tocopherol)'])]

In [15]:
# Definition functions 

def return_household(row):
    household = np.array([])
    for j in range(len(row)):
        if row[j] == 1:
            if (row.index[j] != 'M 0-3') & (row.index[j] != 'F 0-3'):
                household = np.append(household,row.index[j])
    return household

def return_nutrition(household):
    nut_pd = pd.DataFrame(np.array(np.zeros(len(diet_req['Nutrition']))),index=diet_req['Nutrition'])
    for i in range(len(household)): 
        req = diet_req.loc[:,household[i]]
        nut_pd['Person'+ str(i)] = req.to_list()
    nut_pd['Sum'] = nut_pd.sum(axis=1)
    return nut_pd

def actual_consumption(row):
    total_nut = np.zeros(11)
    for j in range(len(row)):
        nut = np.array(np.zeros(11))
        if row[j] > 0:
            nut = (fct.loc[q_items.columns[j]].array * int(row[j]))
        total_nut = total_nut + nut[0:11]
    return total_nut

def is_adequate(num):
    household = return_household(z.iloc[num])
    nut_req = return_nutrition(household)
    act_req = actual_consumption(q_items.iloc[num])
    adeq = False
    if nut_req['Sum'].to_list() < act_req:
        adeq = False
    else: 
        adeq = True
    return adeq

In [16]:
adequate = 0
for i in range(len(q_items)):
    if is_adequate(i):
        adequate = adequate + 1

IndexError: single positional indexer is out-of-bounds